In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: mydemoworkspace
Azure region: westus
Subscription id: d92bf716-fa25-4c22-b6fc-a7f2f009c4ff
Resource group: azuremlrg


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "democomputecluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',                                      
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.wait_for_completion(show_output=True)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler

ps = RandomParameterSampling({
    '--C': choice(1.0, 0.1, 0.05),
    '--max_iter': choice(50,100,150)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
training_folder ='./training'
src = ScriptRunConfig(source_directory=training_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output = True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889
Web View: https://ml.azure.com/runs/HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889?wsid=/subscriptions/d92bf716-fa25-4c22-b6fc-a7f2f009c4ff/resourcegroups/azuremlrg/workspaces/mydemoworkspace&tid=4a0bc1c6-1e96-4840-b84f-601c548235ec

Streaming azureml-logs/hyperdrive.txt

[2022-11-19T08:32:07.032415][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-19T08:32:07.5923760Z][SCHEDULER][INFO]Scheduling job, id='HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_0' 
[2022-11-19T08:32:07.7109472Z][SCHEDULER][INFO]Scheduling job, id='HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_1' 
[2022-11-19T08:32:07.8083886Z][SCHEDULER][INFO]Scheduling job, id='HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_2' 
[2022-11-19T08:32:07.9332217Z][SCHEDULER][INFO]Scheduling job, id='HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_3' 
[2022-11-19T08:32:07.901245][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-11-19T0

{'runId': 'HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889',
 'target': 'democomputecluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-19T08:32:06.218442Z',
 'endTimeUtc': '2022-11-19T08:54:38.913348Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0cc28972-b3c1-41cc-abf5-b5b4e80e8810',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '9',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlC

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

Best Run Id:  HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_0

 Accuracy: 0.9088012139605463


KeyError: 3

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
best_run.get_file_names()
model = best_run.register_model(model_name='hyperdrive_modelV1', model_path='outputs/model.joblib')

Best Run Id:  HD_1d93f3df-a8d7-478f-ada5-6c8d8711d889_0

 Accuracy: 0.9088012139605463


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(example_data)

### YOUR CODE HERE ###

In [12]:
import sys
sys.path.insert(1, './training')
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

datafinal = pd.concat([x,y], axis = 1)
datafinal.head()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data=ds,
    label_column_name= 'y',
    n_cross_validations=5)

In [15]:
# Submit your automl run
exp = Experiment(workspace=ws, name="udacity_automl")
automl_run = exp.submit(config = automl_config, show_output = True)

### YOUR CODE HERE ###

Submitting remote run.
No run_configuration provided, running on democomputecluster with default configuration
Running on remote compute: democomputecluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity_automl,AutoML_65910587-93e6-4c5a-ab3b-f04faaa5b420,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [17]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()

original_model = automl_run.register_model(model_name='automl_model')

### YOUR CODE HERE ###

Package:azureml-automl-runtime, training version:1.46.1, current version:1.44.0
Package:azureml-dataprep, training version:4.5.7, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.11.4, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.46.0, current version:1.44.0
Package:azureml-defaults, training version:1.46.0, current version:1.44.0
Package:azureml-interpret, training version:1.46.0, current version:1.44.0
Package:azureml-mlflow, training version:1.46.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.46.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.46.0, current version:1.44.0
Package:azureml-telemetry, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.46.1, current version:1.44.0
Package:azureml-train-core, training version:1.46.0, current ve